In [1]:
from constants import *
from utils import *

In [2]:
encode_features()

Reading the cleaned data from the database /home/airflow/dags/Lead_scoring_data_pipeline/lead_scoring_data_cleaning.db
Reading Complete!


In [3]:
get_trained_model()

Reading the training_features and training_target from the database /home/airflow/dags/Lead_scoring_data_pipeline/lead_scoring_data_cleaning.db


Registered model 'LightGBM' already exists. Creating a new version of this model...
2024/02/06 13:35:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LightGBM, version 4
Created version '4' of model 'LightGBM'.
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Inside MLflow Run with id f4e7f672c656442aaa1cba29cd217d28


In [ ]:

import pandas as pd
import numpy as np

import sqlite3
from sqlite3 import Error

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#from Lead_scoring_training_pipeline.constants import *

from constants import *

In [ ]:
db_file_path = DB_PATH + DB_FILE_NAME    
db_conn = None    

In [ ]:
print(f"Reading the cleaned data from the database {db_file_path}")
db_conn = sqlite3.connect(db_file_path)
cleaned_data = pd.read_sql('SELECT * FROM model_input', con=db_conn)
db_conn.close()
db_conn = None


In [ ]:
data = cleaned_data.copy()

In [ ]:
data.head()

In [ ]:
ONE_HOT_ENCODED_FEATURES

In [ ]:
FEATURES_TO_ENCODE

In [ ]:
for f in FEATURES_TO_ENCODE:
    if(f in data.columns):
        encoded = pd.get_dummies(data[f])
        encoded = encoded.add_prefix(f + '_')
        data = pd.concat([data, encoded], axis=1)
    else:
        print('Feature not found')


In [ ]:
print(data.columns)

In [ ]:
data.drop(columns=FEATURES_TO_ENCODE, inplace = True)

In [ ]:
data[ONE_HOT_ENCODED_FEATURES]

In [ ]:
data_subset = data[ONE_HOT_ENCODED_FEATURES]

In [ ]:
target_column = data_subset.pop('app_complete_flag')
data_subset

In [ ]:
target_column.head()

In [ ]:
data_subset.head()

In [ ]:
############# -------------------------------

In [ ]:

db_file_path = DB_PATH + DB_FILE_NAME
db_conn = None

In [ ]:
print(f"Reading the training_features and training_target from the database {db_file_path}")
db_conn = sqlite3.connect(db_file_path)

features = pd.read_sql('SELECT * FROM training_features', con=db_conn)
target_col = pd.read_sql('SELECT * FROM training_target', con=db_conn)

db_conn.close()
db_conn = None

X_train, X_test, y_train, y_test = train_test_split(features, target_col, test_size = 0.3, random_state=42)

#mlflow.set_tracking_uri(TRACKING_URI)

In [ ]:
features.head()

In [ ]:
lightgbm_model = lgb.LGBMClassifier()
lightgbm_model.set_params(**model_config)                   # model_config from Constants.py